In [1]:
import pandas as pd
import numpy as np
from itertools import compress

In [2]:
def tsv_to_dataframe(file_tsv):
    '''
    this function transforms Eurostat tsv file in pandas dataframe
    file_tsv: file name. It's work with tsv and compressed file "tsv.gz"
    '''
    
    def clean_cells(x):
        '''This function transforms Eurostat Missing Values ": " in numpy missing values.
        Then clean Eurostat annotation "b, u, .."'''
        try:
            return float(x)
        except:
            try:
                return float(x.split(" ")[0])
            except:
                return np.nan
    # open the Eurostat TSV file 

    data = pd.read_csv(file_tsv, sep="\t")
    # Create a dataframe for values data
    data_clean = data
    # Clean data values with clean_cells function
    data_clean = data_clean.applymap(lambda x: clean_cells(x))
    # Drop column with variable name like "age,isced11,unit,sex,geo\time". It is the first column. we have a 
    # dataframe with only data values 
    data_clean.drop(data_clean.columns[0], axis = 1, inplace = True)
    # transform column with variable in multiple-columns  
    variabili = data[data.columns[0]].apply(lambda x: pd.Series(x.split(",")))
    variabili.columns = data.columns[0].split(",")
    # return cleaned dataframe in pandas dataframe
    return pd.concat([variabili, data_clean], axis = 1)

In [3]:
def tsv_to_dataframe_long(file_tsv):
    '''
    this function transforms Eurostat tsv file in pandas dataframe
    file_tsv: file name. It's work with tsv and compressed file "tsv.gz"
    '''
    
    def clean_cells(x):
        '''This function transforms Eurostat Missing Values ": " in numpy missing values.
        Then clean Eurostat annotation "b, u, .."'''
        try:
            return float(x)
        except:
            try:
                return float(x.split(" ")[0])
            except:
                return np.nan
            
    def annotation(x):
            '''This function extracts Eurostat annotation "b, u, .."'''
            try:
                return x.split(" ")[1]
            except:
                return np.nan

    def columns_type(x):
            try:
                return int(x.strip())
            except:
                if x == "geo\\time":
                    return "var_" + "geo"
                else:
                    return "var_" + x

    
    # open the Eurostat TSV file 

    data = pd.read_csv(file_tsv, sep="\t")
    # Create a dataframe for values data
    data_clean = data
    # Clean data values with clean_cells function
    data_clean = data_clean.applymap(lambda x: x)
    # Drop column with variable name like "age,isced11,unit,sex,geo\time". It is the first column. we have a 
    # dataframe with only data values 
    data_clean.drop(data_clean.columns[0], axis = 1, inplace = True)
    # transform column with variable in multiple-columns  
    variabili = data[data.columns[0]].apply(lambda x: pd.Series(x.split(",")))
    variabili.columns = data.columns[0].split(",")
    # return cleaned dataframe in pandas dataframe
    result = pd.concat([variabili, data_clean], axis = 1)
    colonne  = list(map(lambda x: columns_type(x), list(result.columns.values)))
    result.columns = colonne
    colonne_var = result.columns.map(lambda x: str(x)[0:3] == "var")
    index = list(compress(result.columns, colonne_var))
    result = result.melt(id_vars = index)
    result["value_raw"] = result["value"]
    result["eurostat_annotation"] = result["value_raw"].apply(lambda x: annotation(x))
    result["value"] = result["value"].apply(clean_cells) 
    
    
    return result

In [4]:
#example 
#if __name__ == "__main__":
import urllib.request
import gzip

eurostat_link = "http://ec.europa.eu/eurostat/estat-navtree-portlet-prod/BulkDownloadListing?sort=1&file=data%2Fcrim_off_cat.tsv.gz"
urllib.request.urlretrieve(eurostat_link , "crim_off_cat.tsv.gz")

('crim_off_cat.tsv.gz', <http.client.HTTPMessage at 0x7f0f9b233240>)

In [5]:
df = tsv_to_dataframe_long('crim_off_cat.tsv.gz')
df

,var_iccs,var_unit,var_geo,variable,value,value_raw,eurostat_annotation
0,ICCS0101,NR,AL,2015,54.00,54,
1,ICCS0101,NR,AT,2015,42.00,42,
2,ICCS0101,NR,BA,2015,56.00,56,
3,ICCS0101,NR,BE,2015,220.00,220,
4,ICCS0101,NR,BG,2015,129.00,129,
5,ICCS0101,NR,CH,2015,57.00,57,
6,ICCS0101,NR,CY,2015,12.00,12,
7,ICCS0101,NR,CZ,2015,84.00,84,
8,ICCS0101,NR,DE,2015,655.00,655,
9,ICCS0101,NR,DK,2015,46.00,46,


In [6]:
for colonne in df.columns:
    print(df[colonne].unique())
    

['ICCS0101' 'ICCS0102' 'ICCS02011' 'ICCS020221' 'ICCS0301' 'ICCS03011'
 'ICCS03012' 'ICCS0401' 'ICCS0501' 'ICCS05012' 'ICCS0502' 'ICCS050211'
 'ICCS0601']
['NR' 'P_HTHAB']
['AL' 'AT' 'BA' 'BE' 'BG' 'CH' 'CY' 'CZ' 'DE' 'DK' 'EE' 'EL' 'ES' 'FI'
 'FR' 'HR' 'HU' 'IE' 'IS' 'IT' 'LI' 'LT' 'LU' 'LV' 'ME' 'MK' 'MT' 'NL'
 'NO' 'PL' 'PT' 'RO' 'RS' 'SE' 'SI' 'SK' 'TR' 'UKC-L' 'UKM' 'UKN' 'XK']
[2015 2014 2013 2012 2011 2010 2009 2008]
[ 54.    42.    56.   ... 198.9   34.29  14.58]
['54 ' '42 ' '56 ' ... '198.90 ' '34.29 ' '14.58 ']
['']


In [7]:
df["var_iccs"][0]=="ICCS0101"

True

In [17]:
cond1=df["var_iccs"]=="ICCS0101"
cond2=df["var_iccs"]=="ICCS02011"
cond3=df["var_iccs"]=="ICCS0301"
cond4=df["var_iccs"]=="ICCS03011"

In [18]:
reatigravi = df[cond1 | cond2 | cond3 | cond4]
reatigravi

,var_iccs,var_unit,var_geo,variable,value,value_raw,eurostat_annotation
0,ICCS0101,NR,AL,2015,54.00,54,
1,ICCS0101,NR,AT,2015,42.00,42,
2,ICCS0101,NR,BA,2015,56.00,56,
3,ICCS0101,NR,BE,2015,220.00,220,
4,ICCS0101,NR,BG,2015,129.00,129,
5,ICCS0101,NR,CH,2015,57.00,57,
6,ICCS0101,NR,CY,2015,12.00,12,
7,ICCS0101,NR,CZ,2015,84.00,84,
8,ICCS0101,NR,DE,2015,655.00,655,
9,ICCS0101,NR,DK,2015,46.00,46,


In [19]:
for colonne in reatigravi.columns:
    print(reatigravi[colonne].unique())

['ICCS0101' 'ICCS02011' 'ICCS0301' 'ICCS03011']
['NR' 'P_HTHAB']
['AL' 'AT' 'BA' 'BE' 'BG' 'CH' 'CY' 'CZ' 'DE' 'DK' 'EE' 'EL' 'ES' 'FI'
 'FR' 'HR' 'HU' 'IE' 'IS' 'IT' 'LI' 'LT' 'LU' 'LV' 'ME' 'MK' 'MT' 'NL'
 'NO' 'PL' 'PT' 'RO' 'RS' 'SE' 'SI' 'SK' 'TR' 'UKC-L' 'UKM' 'UKN' 'XK']
[2015 2014 2013 2012 2011 2010 2009 2008]
[54.   42.   56.   ... 23.98 15.83  2.32]
['54 ' '42 ' '56 ' ... '23.98 ' '15.83 ' '2.32 ']
['']


In [24]:
pd.pivot_table(reatigravi, values = "value",
               index = "var_iccs",
               aggfunc = np.mean)

,value
var_iccs,
ICCS0101,90.142436
ICCS02011,16311.787460
ICCS0301,2860.134355
ICCS03011,829.912360


In [21]:
pd.pivot_table(reatigravi, values = "value",
               index = ["var_geo","var_iccs"],
               aggfunc = [np.mean,np.std]).T

var_geo            AL                                           AT  \
var_iccs     ICCS0101   ICCS02011   ICCS0301  ICCS03011   ICCS0101   
mean value  73.248182  158.029091  55.575455  25.643636  30.483750   
std  value  49.373333  125.756997  45.308520  18.495160  32.837244   

var_geo                                                  BA              \
var_iccs      ICCS02011     ICCS0301   ICCS03011   ICCS0101   ICCS02011   
mean value  1918.545000  1918.608750  559.820000  34.976154  499.505455   
std  value  1940.317161  1943.260498  587.534098  28.171624  464.737978   

var_geo       ...              UKM                    UKN              \
var_iccs      ...         ICCS0301   ICCS03011   ICCS0101   ICCS02011   
mean value    ...      3218.625714  636.390714  11.871875  712.319375   
std  value    ...      3372.289334  687.861574  11.229724  671.058154   

var_geo                                     XK                           \
var_iccs       ICCS0301   ICCS03011   ICCS0101    ICCS02011    ICCS0301   
mean value  1097.619375  277.416250  47.571667  1781.248333  104.938333   
std  value  1053.706034  268.109514  40.784394  1669.112236   76.649678   

var_geo                
var_iccs    ICCS03011  
mean value  24.097500  
std  value  17.759004  

[2 rows x 159 columns]

In [29]:
cond = reatigravi["var_geo"].isin(["IT","ES","DE","FR"])
eurogravi=reatigravi[cond]

In [30]:
pd.pivot_table(eurogravi, values = "value",
               index = ["var_geo","var_iccs"],
               aggfunc = [np.mean,np.std]).T

var_geo             DE                                                    ES  \
var_iccs      ICCS0101      ICCS02011      ICCS0301    ICCS03011    ICCS0101   
mean value  332.096250   91793.036250  20000.779375  3734.285625  181.451875   
std  value  343.048845  132412.302564  21373.379290  3852.963731  189.290852   

var_geo                                                    FR                 \
var_iccs       ICCS02011     ICCS0301   ICCS03011    ICCS0101      ICCS02011   
mean value  17347.808750  4868.276875  790.764375  429.533125  113822.586875   
std  value  25430.618166  5024.107448  867.515831  447.774255  117788.422168   

var_geo                                        IT                
var_iccs        ICCS0301    ICCS03011    ICCS0101     ICCS02011  
mean value  13306.549375  5496.039375  266.946875  33232.671250  
std  value  13939.908836  5707.085324  277.032888  34230.096778

In [36]:
eurogravi.groupby("var_geo").mean()

,value
var_geo,
DE,28965.049375
ES,5797.075469
FR,33263.677188
IT,16749.809063


In [46]:
eurogravi.columns

Index(['reati', 'var_unit', 'var_geo', 'variable', 'value', 'value_raw',
       'eurostat_annotation'],
      dtype='object')

In [47]:
eurogravi.columns = ['reati', 'var_unit', 'nazione', 'anno', 'valore', 'value_raw',
       'eurostat_annotation']

In [53]:
def cambionome(reato):
    if reato == "ICCS0101":
        return "omicidio intenzionale"
    if reato == "ICCS02011":
        return "aggressione"
    if reato == "ICCS0301":
        return "violenza sessuale"
    if reato == "ICCS03011":
        return "stupro"

In [55]:
eurogravi["reati"]=eurogravi["reati"].apply(cambionome)

/anaconda/envs/py35/lib/python3.5/site-packages/ipykernel_launcher.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  """Entry point for launching an IPython kernel.


In [61]:
eurogravi=eurogravi.reset_index(drop=True)